# Sentinel‑2 Cloud & Shadow Masking Workflow (s2cloudless)
This notebook builds a cloud‑free composite, computes NDVI & SWI, and overlays Dynamic World grass and SWI outlines, all using Earth Engine.
Run each cell top‑to‑bottom. You can tweak the user‑input section to analyse a different AOI or date range.

Follow https://developers.google.com/earth-engine/reference/Quickstart#before-you-begin to get a secret key

In [37]:
import ee, geemap, datetime, json

KEY = 'my-secret-key.json'
with open(KEY, 'r') as f:
    data = json.load(f)
    SERVICE_ACCOUNT = data['client_email']
    PROJECT = data['project_id']

ee_creds = ee.ServiceAccountCredentials(SERVICE_ACCOUNT, KEY)
ee.Initialize(ee_creds)

## User inputs – AOI, dates, thresholds

In [38]:
geom = ee.Geometry.BBox(36.2597202470, 4.19477694745,
                 36.3308408646, 4.26022461625)
time_start  = '2025-05-14'
time_end    = '2025-06-14'

cloud_thresh = 60   # % scene‑level filter
CLD_PRB_THRESH = 30 # s2cloudless probability threshold
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST   = 2  # km
BUFFER         = 100  # m
ndvi_thresh  = 0.3
swi_thresh   = 0.2

## Helper functions – cloud & shadow masking, NDVI, timestamp

In [39]:
def get_s2_sr_cld_col(aoi, start_date, end_date):
    s2_sr = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
              .filterBounds(aoi)
              .filterDate(start_date, end_date)
              .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', cloud_thresh)))
    s2_cld = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
              .filterBounds(aoi)
              .filterDate(start_date, end_date))
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr,
        'secondary': s2_cld,
        'condition': ee.Filter.equals(leftField='system:index', rightField='system:index')
    }))

def add_cloud_bands(img):
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')
    return img.addBands([cld_prb, is_cloud])

def add_shadow_bands(img):
    not_water = img.select('SCL').neq(6)
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH * 1e4).multiply(not_water).rename('dark_pixels')
    azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')))
    cld_proj = (img.select('clouds').directionalDistanceTransform(azimuth, CLD_PRJ_DIST * 10)
                 .reproject(crs=img.select(0).projection(), scale=100)
                 .select('distance').mask().rename('cloud_transform'))
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')
    return img.addBands([dark_pixels, cld_proj, shadows])

def add_cld_shdw_mask(img):
    img_cloud = add_cloud_bands(img)
    img_cloud_shadow = add_shadow_bands(img_cloud)
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
                    .reproject(crs=img.select([0]).projection(), scale=20)
                    .rename('cloudmask'))
    return img_cloud_shadow.addBands(is_cld_shdw)

def apply_cld_shdw_mask(img):
    return img.updateMask(img.select('cloudmask').Not())

def add_ndvi(img):
    return img.addBands(img.normalizedDifference(['B8', 'B4']).rename('NDVI'))

def add_timestamp(img):
    ts = ee.Image.constant(img.date().millis()).rename('timestamp').toInt64()
    return img.addBands(ts)

def preprocess(img):
    """Add NDVI, timestamp (after mask) and a numeric image‑ID band."""
    # ---------- NDVI
    img = img.addBands(img.normalizedDifference(['B8','B4']).rename('NDVI'))
    
    # ---------- timestamp band (ms since 1970‑01‑01)
    ts = ee.Image.constant(img.date().millis()) \
            .rename('timestamp') \
            .toInt64() \
            .updateMask(img.select('B8').mask())      # keep only valid pixels
    img = img.addBands(ts)
    
    # ---------- numeric image ID (0,1,2,…) so you can colour by source
    img_id = ee.Image.constant(img.get('system:time_start')) \
                .toInt64() \
                .rename('img_id') \
                .updateMask(img.select('B8').mask())
    return img.addBands(img_id)

## Build cloud‑masked collection & composite

In [40]:
col_raw = get_s2_sr_cld_col(geom, time_start, time_end)
col_clean = (col_raw
             .map(add_cld_shdw_mask)
             .map(apply_cld_shdw_mask)
             .map(preprocess))

filled   = col_clean.qualityMosaic('timestamp').clip(geom)
rgb      = filled.select(['B4','B3','B2']).divide(1e4)
timestamp_band = filled.select('timestamp')
ndvi   = filled.select('NDVI')
swi    = filled.normalizedDifference(['B8','B11']).rename('SWI').gt(swi_thresh)
newest_raw = ee.Image(col_raw.sort('system:time_start', False).first()).clip(geom)
newest_raw = newest_raw.select(['B4','B3','B2']).divide(1e4)

## Dynamic World grass mask & outline

In [41]:
dw = (ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1')
        .filterBounds(geom)
        .filterDate(time_start, time_end)
        .qualityMosaic('grass')
        .clip(geom))
dw_grass = dw.select('grass').gt(0.3)
dw_outline = (dw_grass.reduceToVectors(
        geometry=geom,
        scale=50,
        geometryType='polygon',
        eightConnected=False,
        labelProperty='grass',
        reducer=ee.Reducer.countEvery())
        .filter(ee.Filter.gt('grass', 0))
        .map(lambda f: f.buffer(30).simplify(30))
        .map(lambda f: f.set('area', f.geometry().area()))
        .filter(ee.Filter.gt('area', 1e5)))
grass_mask  = ee.Image(0).byte().paint(dw_outline, 1)
ndvi_masked = ndvi.updateMask(grass_mask).updateMask(ndvi.gt(ndvi_thresh))

## SWI > 0.2 outline

In [42]:
swi_outline = (swi.reduceToVectors(
        geometry=geom,
        scale=50,
        geometryType='polygon',
        eightConnected=True,
        labelProperty='SWI',
        reducer=ee.Reducer.countEvery())
        .filter(ee.Filter.gt('SWI', 0))
        .map(lambda f: f.buffer(30).simplify(30))
        .map(lambda f: f.set('area', f.geometry().area())))

## NDVI > 0.3 Polygon

In [43]:
# mask out NDVI below threshold
ndvi_gt = ndvi.gt(0.3).rename('ndvi_gt')

ndvi_outline = (ndvi_gt.updateMask(ndvi.gt(0.3)).reduceToVectors(
        geometry=geom,
        scale=50,
        geometryType='polygon',
        eightConnected=True,
        labelProperty='NDVI',
        reducer=ee.Reducer.countEvery())
        .filter(ee.Filter.gt('NDVI', 0))
        .map(lambda f: f.buffer(30).simplify(30))
        .map(lambda f: f.set('area', f.geometry().area()))
        .filter(ee.Filter.gt('area', 1e3)))
        

## Visualise layers on an interactive map

In [44]:
Map = geemap.Map()
Map.centerObject(geom, 12)
rgb_vis = {'min':0, 'max':0.3}
ndvi_vis  = {'min':0,'max':1,'palette':['red','yellow','green']}
ts_vis = {'min':ee.Date(time_start).millis().getInfo(),
          'max':ee.Date(time_end).millis().getInfo(),
          'palette':['red','yellow','green']}
dw_vis = {'min':0,'max':1,'palette':['white','green']}
swi_vis = {'min':0,'max':1,'palette':['white','blue']}
ts_vis  = {          
    'min': ee.Date(time_start).millis().getInfo(),
    'max': ee.Date(time_end).millis().getInfo(),
    'palette': ['red', 'yellow', 'green']
}

Map.addLayer(newest_raw, rgb_vis, 'RGB (Newest Raw)')
Map.addLayer(rgb, rgb_vis, 'RGB Composite')
Map.addLayer(timestamp_band, ts_vis, 'Pixel acquisition date (older→red, newer→green)')
Map.addLayer(ndvi, ndvi_vis, 'NDVI (Masked)')
Map.addLayer(dw.select('grass'), dw_vis, 'Dynamic World Grass')
Map.addLayer(dw_outline, {'color':'blue'}, 'Grass Outline')
Map.addLayer(ndvi_masked, ndvi_vis, 'NDVI Masked to Grass')
Map.addLayer(swi, swi_vis, 'SWI > 0.2')
Map.addLayer(swi_outline, {'color':'orange'}, 'SWI Outline')
Map.addLayer(ndvi_outline, {'color':'purple'}, 'NDVI Outline')
Map

Map(center=[4.227500593872711, 36.29528055580051], controls=(WidgetControl(options=['position', 'transparent_b…

## Acquisition dates used in this composite

In [17]:
dates = col_clean.aggregate_array('system:time_start').getInfo()
dates = [datetime.datetime.utcfromtimestamp(t/1000).strftime('%Y-%m-%d') for t in dates]
for d in dates:
    print('•', d)

• 2025-05-15
• 2025-05-15
• 2025-05-20
• 2025-05-20
• 2025-05-25
• 2025-05-25
• 2025-05-30
• 2025-05-30
• 2025-06-01
• 2025-06-01
• 2025-06-04
• 2025-06-04
• 2025-06-09
• 2025-06-09
• 2025-06-11
• 2025-06-11
